# Analytic queries
## Some introductory queries against sparkify to help the analysts identify trends

1. Most popular songs
2. Locations which listened to the most songs
3. Top user agents
4. Top users
5. Average number of songs per session
6. Count of songplays grouped by pricing tier (level)

## Imports

In [1]:
import configparser

import pandas as pd
import pandas.io.sql as sqlio
import psycopg2

## Get credentials and connect to Redshift cluster

In [2]:
config = configparser.ConfigParser()
config.read('../dwh.cfg')
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [3]:
def query(sql):
    """Executes a sql query and builds a dataframe
    :param sql: The SQL statement to be executed
    :param return: pandas.core.frame.DataFrame
    """
    return sqlio.read_sql_query(sql, conn)

## Sample data

In [4]:
query("select * FROM songplays limit 5")

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,16,2018-11-05 15:09:05,24,paid,None,None,23,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,80,2018-11-03 18:09:31,15,paid,None,None,199,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
2,144,2018-11-08 08:36:13,83,free,None,None,258,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,208,2018-11-13 08:49:43,25,paid,None,None,442,"Marinette, WI-MI","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
4,272,2018-11-13 19:46:43,29,paid,None,None,556,"Atlanta-Sandy Springs-Roswell, GA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


## Analytics queries

### Most popular songs

In [5]:
query("""
select 
    song_id,
    count(*) num_plays
from songplays
where song_id is not null
group by 1 order by 2 desc
limit 10
""")

,song_id,num_plays
0,SOBONKR12A58A7A7E0,37
1,SOUNZHU12A8AE47481,9
2,SOHTKMO12AB01843B0,9
3,SOULTKQ12AB018A183,8
4,SOLZOBD12AB0185720,6
5,SOARUPP12AB01842E0,5
6,SOTNHIP12AB0183131,5
7,SOIZLKI12A6D4F7B61,4
8,SONQEYS12AF72AABC9,4
9,SOIOESO12A6D4F621D,4


### Locations which listened to the most songs

In [6]:
query("""
select 
    location,
    count(*) number_of_plays
from songplays
where location is not null
group by 1 order by 2 desc
limit 10
""")

,location,number_of_plays
0,"San Francisco-Oakland-Hayward, CA",691
1,"Portland-South Portland, ME",665
2,"Lansing-East Lansing, MI",557
3,"Chicago-Naperville-Elgin, IL-IN-WI",475
4,"Atlanta-Sandy Springs-Roswell, GA",456
5,"Waterloo-Cedar Falls, IA",397
6,"Lake Havasu City-Kingman, AZ",321
7,"Tampa-St. Petersburg-Clearwater, FL",307
8,"San Jose-Sunnyvale-Santa Clara, CA",292
9,"Sacramento--Roseville--Arden-Arcade, CA",270


### Top user agents

In [7]:
query("""
select 
    user_agent,
    count(*) num_users
from songplays
where user_agent is not null
group by 1 order by 2 desc
limit 10
""")

,user_agent,num_users
0,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",971
1,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",708
2,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,696
3,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",577
4,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",573
5,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,443
6,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",427
7,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",419
8,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",319
9,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,310


### Top users

In [8]:
query("""
select 
    user_id,
    count(*) num_songs_played
from songplays
where user_id is not null
group by 1 order by 2 desc
limit 10
""")

,user_id,num_songs_played
0,49,689
1,80,665
2,97,557
3,15,463
4,44,397
5,29,346
6,24,321
7,73,289
8,88,270
9,36,248


### Average number of songs per session

In [9]:
query("""
select avg(num_plays) average_listens_per_session
from
    (select 
        session_id,
        count(*) num_plays
    from songplays
    group by 1)
""")

,average_listens_per_session
0,8


### Count of songplays grouped by pricing tier (level)

In [10]:
query("""
select level, count(*) songplays_per_tier
from songplays
group by 1
""")

,level,songplays_per_tier
0,paid,5591
1,free,1229
